In [ ]:
#_________________________________________________________________________________________________________________________#
#______________________________________________________Raven.ai___________________________________________________________#
#_________________________________________________________________________________________________________________________#
import tensorflow as tf
from tensorflow.keras.datasets import mnist
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
# Let's declare some const here and use them all the way down, we are tackling MNIST dataset, 
# a classical one. it contain images of handwritten digits from 0 to 9, image dimensions are 28X28 balck and white colors 

num_features = 784 #28*28
batch_size = 256 #number of samples to feed our model
num_classes = 10 #
learning_rate = 0.01
training_steps = 2000 #number of iteartions 

# Loading data here !!!

(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Convert to float32.

x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)

# Flatten images to 1-D vector of 784 features (28*28).

x_train, x_test = x_train.reshape([-1, num_features]), x_test.reshape([-1, num_features])

# Normalize images value from [0, 255] to [0, 1].

x_train, x_test = x_train / 255., x_test / 255.

11501568/11490434 [==============================] - 0s 0us/step


In [ ]:
train_data=tf.data.Dataset.from_tensor_slices((x_train,y_train))

train_data=train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

In [ ]:
# Weight of shape [784, 10], the 28*28 image features, and a total number of classes.

W = tf.Variable(tf.ones([num_features, num_classes]), name="weight")

# Bias of shape [10], the total number of classes.

b = tf.Variable(tf.zeros([num_classes]), name="bias")

In [ ]:
def logistic_regression(x):

    # Apply softmax to normalize the logits to a probability distribution.

    return tf.nn.softmax(tf.matmul(x, W) + b)

# Cross-Entropy loss function.

def cross_entropy(y_pred, y_true):

    # Encode label to a one hot vector.

    y_true = tf.one_hot(y_true, depth=num_classes)

    # Clip prediction values to avoid log(0) error.

    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)

    # Compute cross-entropy.

    return tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred)))

In [ ]:
# Accuracy metric.

def accuracy(y_pred, y_true):

# Predicted class is the index of the highest score in prediction vector (i.e. argmax).

  correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))

  return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Stochastic gradient descent optimizer.

optimizer = tf.optimizers.SGD(learning_rate)

In [ ]:
def run_optimization(x, y):

# Wrap computation inside a GradientTape for automatic differentiation.

    with tf.GradientTape() as g:

        pred = logistic_regression(x)

        loss = cross_entropy(pred, y)

    # Compute gradients.

    gradients = g.gradient(loss, [W, b])

  

    # Update W and b following gradients.

    optimizer.apply_gradients(zip(gradients, [W, b]))

In [ ]:
# Run training for the given number of steps.

epochs = []
metrics = []
loss = []


for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):

    # Run the optimization to update W and b values.

    run_optimization(batch_x, batch_y)

    pred = logistic_regression(batch_x)

    cost = cross_entropy(pred, batch_y)

    acc = accuracy(pred, batch_y)

    epochs.append(step)

    metrics.append(acc)

    loss.append(cost)

    if step % 250 == 0:
      print("step: %i, loss: %f, accuracy: %f" % (step, cost, acc))

# Plotting all that metrics 

# acc plot
fig = go.Figure()
accuracies = fig.add_trace(go.Scatter(x=epochs, y=metrics,
                    mode='lines',
                    name='acc',
                    ))
accuracies.add_annotation(x=500, y=1.2, showarrow=False, text="<b>ACC</b>")
accuracies.show()


# loss plot
fig = go.Figure()
losses = fig.add_trace(go.Scatter(x=epochs, y=loss,
                    mode='lines',
                    name='loss',
                    marker_color='rgba(152, 0, 0, .8)'
                    ))
losses.add_annotation(x=500, y=1200, showarrow=False, text="<b>LOSS</b>")
losses.show()



    

step: 250, loss: 524.705017, accuracy: 0.867188
step: 500, loss: 560.615967, accuracy: 0.820312
step: 750, loss: 481.301453, accuracy: 0.878906
step: 1000, loss: 525.352478, accuracy: 0.875000
step: 1250, loss: 610.074219, accuracy: 0.871094
step: 1500, loss: 732.645386, accuracy: 0.832031
step: 1750, loss: 487.523468, accuracy: 0.863281
step: 2000, loss: 523.436523, accuracy: 0.871094


In [ ]:
# Test model on validation set.

pred = logistic_regression(x_test)

print("Test Accuracy: %f" % accuracy(pred, y_test))

Test Accuracy: 0.839700
